# <span style="color:#0485CF"> Conexión a Base de Datos </span>


### Instalación del manejador de Base de Datos: PostgreSQL

#### https://www.postgresql.org

- PostgreSQL, comúnmente pronunciado "Post-GRES"
- Es una base de datos de código abierto que tiene una sólida reputación por su fiabilidad, flexibilidad y soporte de estándares técnicos abiertos.
- PostgreSQL está completamente libre de restricciones de licencia

###  Instalación 

#### 1. Descargar  el manejador de Base de Datos: PostgreSQL

#### 2. Instalación del manejador de Base de Datos: PostgreSQL

#### 3. Configurar el path 



### Creación de ambiente para PostgreSQL

- En la terminal colocar  los siguientes comandos


#### Crear el ambiente desde la terminal

In [ ]:
conda create --name  PostSQL python

#### Activar el ambiente desde la terminal

In [ ]:
conda activate PostSQL

### Instalación de la GUI (Encaso de que no se encuentre en la instalación de PostgreSQL)

In [ ]:
pip install pgadmin4

### Instalación de paquetes para PosgreSQL en el ambiente creado

#### Desde la terminal

In [ ]:
pip install psycopg2
pip install psycopg2-binary

### Crear un usuario de conexion

### Desde el prompt de la terminal: conectarse al Manejador de BD de PostgreSQL
- En el caso de linux, mac os: desde la terminal
- En el caso de windows: desde la terminal del prompt de PostgreSQL  (seguir las instrucciones en pantalla)

In [ ]:
psql -U postgres

### Desde el prompt de postgreSQL

In [ ]:

CREATE USER uacm WITH PASSWORD 'uacm1';


In [ ]:
# comando para listar os usuarios 
\du

In [ ]:
# Privilegios para crear BD
ALTER USER uacm CREATEDB;

In [ ]:
#c Cambiarse al usuario creado para crear BDs o 
SET ROLE uacm;

In [ ]:
# Salir de psql 
exit

### Crear una base de datos

### Desde el prompt de la terminal: conectarse al Manejador de BD con el usuario creado
- En el caso de linux, mac os: desde la terminal
- En el caso de windows: desde la terminal del prompt de PostgreSQL (seguir las instrucciones en pantalla: cambiar el usuario)

In [ ]:
# Desde la terminal 
psql -U uacm postgres


In [ ]:
CREATE DATABASE sistema_abc
  WITH ENCODING='UTF8'
  LC_COLLATE='es_ES.UTF-8'
  LC_CTYPE='es_ES.UTF-8'
  TEMPLATE=template0;

In [ ]:
# Listar las Bases de Datos disponibles
\l

In [ ]:
# Conectarse a la Base de Datos sistemaABC
\c sistema_abc

### Crear la tabla

In [ ]:
CREATE TABLE productos (
    id SERIAL PRIMARY KEY,
    descripcion TEXT,
    precio NUMERIC(10, 2)
);

In [ ]:
# Listar las tablas

\dt

In [ ]:
# Mostrar detalles de la tabla
\d productos

In [ ]:
# Insertar datos 
INSERT INTO productos (descripcion, precio) VALUES ('camisa', 99.99);

INSERT INTO productos (descripcion, precio) VALUES ('pantalon', 499.99);


In [ ]:
# Seleccionar datos

SELECT * FROM productos;


# Seleccionar datos según un criterio

SELECT * FROM productos WHERE precio < 100;


## Ejercicio

### 1. Crear una tabla de Empleados con los campos:

- Identificador del empleado tipo númerico    
- Nombre del  empleado tipo texto
- Fecha de Nacimiento tipo date
- Puesto del empleado tipo numerico
- Sueldo mensual del empleado tipo numerico

### 2. Insertar 5 datos

### 3. Mostrar los datos de la tabla Empleados

### 4. Mostrar los datos de la tabla Empleados que cumplan con un sueldo menor de 


### <span style="color:red"> Nota: recordar que cada instrucción termina con punto y coma ;</span>





# Conexión a la Base de Datos desde Python

In [ ]:
import psycopg2

def connect():
    try:
        # conectando al servidor de PostgreSQL 
        conn = psycopg2.connect(
                            host="localhost",
                            database="sistema_abc",
                            user="uacm",
                            password="uacm1"
                        )
        print('Conectado con el Servidor de PostgreSQL')
        return conn
    except (psycopg2.DatabaseError, Exception) as error:
        print(error)


conn = connect()

In [ ]:
# Abrir un cursor para hacer operaciones

cur = conn.cursor()
cur.execute("SELECT * FROM productos")
registros = cur.fetchall()
cur.close()

for r in registros:
    idx, desc, precio = r     
    print("identificador: {}".format(idx))
    print("\tdescripción: {}".format(desc))
    print("\tprecio: {}".format(precio))


In [ ]:
# Tipos de datos

type(registros)
print(registros[0])
print(registros[1])

In [ ]:
# insertar datos
cur = conn.cursor()
cur.execute("INSERT INTO productos(descripcion, precio) values ('gorras', 399);")
cur.close()




In [ ]:
# Actualizar  datos

cur = conn.cursor()
cur.execute("UPDATE productos SET precio = 699.99 WHERE   id = 1 ") # camisas
cur.close()


In [ ]:
conn.commit()

# <span style="color:red "> En la Base de datos no se actualiza hasta hacer commit </span>: Consultar en la terminal

In [ ]:

sistema_abc=> SELECT * FROM productos;
  1 | camisa      |  99.99
  2 | pantalon    | 499.99

In [ ]:
conn.commit()

# Consultar en la terminal

In [ ]:
id_registro = 2 
# Ejecutar la consulta para borrar el registro
cur = conn.cursor()
cur.execute(f"DELETE FROM productos WHERE id = {id_registro}")
cur.close()
conn.commit()

# Consultar en la terminal

In [ ]:
sistema_abc=> SELECT * FROM productos;
  1 | camisa      |  99.99
  3 | gorras      | 399.00

# Cerrar la conexión a la Base de Datos

In [ ]:
conn.close()

## Ejercicio

### 1. Crear una clase ConexionDB con las funcionalidades:

- Abrir la conexión
- Cerrar conexión
- Ejecutar  Comando / cursor:
    
    - Consultar 
        - Debe crear el cursor y cerrarlo dentro
        - Debe retornar los datos consultados


- Demostrar el uso de la clase, creando un objeto de tipo ConexionDB y hacer uso de consultar.






# Conexión con archivo de configuración

### Crear un archivo de configuración para la conexión (database.ini)

```
[postgresql]
host=localhost
database=sistema_abc
user=uacm
password=uacm1
```

In [ ]:
from configparser import ConfigParser
import psycopg2

def load_config(filename='database.ini', section='postgresql'):
    parser = ConfigParser()
    parser.read(filename)

    # obtiene la seccion por default postgresql
    config = {}
    if parser.has_section(section):
        for param in parser.items(section):
            config[param[0]] = param[1]
    else:
        raise Exception('La sección  {0} no se encuentra en  {1}'.format(section, filename))

    return config

def connect(**config):
    try:
        # conectando al servidor de PostgreSQL 
        conn = psycopg2.connect(**config)
        print('Conectado con el Servidor de PostgreSQL')
        return conn
    except (psycopg2.DatabaseError, Exception) as error:
        print(error)


config = load_config()
print(config)
conn = connect(**config)

In [ ]:
# Abrir un cursor para hacer operaciones

cur = conn.cursor()
cur.execute("SELECT * FROM productos")
registros = cur.fetchall()
cur.close()

for r in registros:
    idx, desc, precio = r     
    print("identificador: {}".format(idx))
    print("\tdescripción: {}".format(desc))
    print("\tprecio: {}".format(precio))

In [ ]:
conn.close()

## Ejercicio

### 1. Crear una clase ConexionDB con las funcionalidades:

- Abrir la conexión
- Cerrar conexión
- Ejecutar  Comando / cursor:
    
    - Insertar 
    - Actualizar
    - Borrar


- Demostrar el uso de la clase, creando un objeto de tipo ConexionDB y hacer uso los comandos.



